In [2]:
from google.colab import drive
drive.mount('/content/drive')
debug = False

Mounted at /content/drive


In [3]:
import os
os.getcwd()

'/content'

In [4]:
os.chdir("/content/drive/MyDrive/thesis/2_MODELING")

In [5]:
os.getcwd()

'/content/drive/MyDrive/thesis/2_MODELING'

In [6]:
import pandas as pd
train = pd.read_parquet("../files/processed/paired_datasets/train.parquet")



In [7]:
train=train[['vacant_id', 'pos_candidate_id', 'neg_candidate_id','pos_candidate_full_text', 'neg_candidate_full_text','neg_vacant_full_text']]

In [8]:
import random
def pick_random_chunks(job_text, pos_cand_text, neg_cand_text):
    def sample(s):
        l = len(s)
        if l <= 512:
            return s
        i = random.randint(0, l - 512)
        return s[i:i+512]

    return {
        "anchor":   sample(job_text),
        "positive": sample(pos_cand_text),
        "negative": sample(neg_cand_text),
    }
frames=[]
# then:
for i in range(4):
    tr = train[['vacant_id', 'pos_candidate_id', 'neg_candidate_id',
                'pos_candidate_full_text', 'neg_candidate_full_text',
                'neg_vacant_full_text']].copy()

    tr[['anchor', 'positive', 'negative']] = tr.apply(
        lambda r: pick_random_chunks(
            r['neg_vacant_full_text'],          # job
            r['pos_candidate_full_text'],       # pos cv
            r['neg_candidate_full_text'],       # neg cv
        ),
        axis=1,
        result_type="expand"
    )

    tr = tr[['vacant_id', 'pos_candidate_id', 'neg_candidate_id',
             'anchor', 'positive', 'negative']]
    frames.append(tr)


In [9]:
train = pd.concat(frames)

In [ ]:
train

In [11]:
val = pd.read_parquet("../files/processed/paired_datasets/val.parquet")
val=val[['vacant_id', 'pos_candidate_id', 'neg_candidate_id','pos_candidate_full_text', 'neg_candidate_full_text','neg_vacant_full_text']]


In [ ]:
val

In [13]:
frames=[]
for i in range(4):
    tr = val[['vacant_id', 'pos_candidate_id', 'neg_candidate_id',
                'pos_candidate_full_text', 'neg_candidate_full_text',
                'neg_vacant_full_text']].copy()

    tr[['anchor', 'positive', 'negative']] = tr.apply(
        lambda r: pick_random_chunks(
            r['neg_vacant_full_text'],          # job
            r['pos_candidate_full_text'],       # pos cv
            r['neg_candidate_full_text'],       # neg cv
        ),
        axis=1,
        result_type="expand"
    )

    tr = tr[['vacant_id', 'pos_candidate_id', 'neg_candidate_id',
             'anchor', 'positive', 'negative']]
    frames.append(tr)


In [ ]:
val = pd.concat(frames)
val

In [15]:
cols ={ "pos_chunks":"positive", 	"neg_chunks":"negative", 	"job_chunks":"anchor"}
train = train.rename(columns = cols)
val = val.rename(columns = cols)


In [16]:
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel



In [17]:
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss, TripletLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = SentenceTransformer(
    "sentence-transformers/all-mpnet-base-v2",
    device=device,          # <--- forces GPU if available
)

Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:

model = SentenceTransformer(
    "./base-best_3/checkpoint-54360", # se ha corrido varias veces
    device=device,
)

In [19]:
from datasets import Dataset, DatasetDict

train_hf = Dataset.from_pandas(
    train[['anchor', 'positive', 'negative']],
    preserve_index=False
)

val_hf = Dataset.from_pandas(
    val[['anchor', 'positive', 'negative']],
    preserve_index=False
)

dataset = DatasetDict({
    "train": train_hf,
    "dev": val_hf
})

dataset


DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 1087128
    })
    dev: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 563380
    })
})

In [20]:
# 3. Load a dataset to finetune on
train_dataset = dataset["train"]
eval_dataset = dataset["dev"]


In [ ]:

loss = MultipleNegativesRankingLoss(model)

args = SentenceTransformerTrainingArguments(
    output_dir="./base-best_3",
    num_train_epochs=8,
    per_device_train_batch_size=100,
    per_device_eval_batch_size=100,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=Fals
    bf16=True,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  
    save_strategy="epoch",
    logging_steps=500000

)




In [27]:

dev_evaluator = TripletEvaluator(
    anchors=eval_dataset["anchor"],
    positives=eval_dataset["positive"],
    negatives=eval_dataset["negative"],
    name="base-mpnet",
)

In [25]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [28]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    evaluator=dev_evaluator,
)
trainer.train(resume_from_checkpoint="./base-best_3/checkpoint-54360")

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: juanjcorredoro (juanjcorredoro-universidad-de-los-andes) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=86976, training_loss=1.1394462445099798, metrics={'train_runtime': 16521.8185, 'train_samples_per_second': 526.396, 'train_steps_per_second': 5.264, 'total_flos': 0.0, 'train_loss': 1.1394462445099798, 'epoch': 8.0})

In [30]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [31]:


test_evaluator = TripletEvaluator(
    anchors=eval_dataset["anchor"],
    positives=eval_dataset["positive"],
    negatives=eval_dataset["negative"],
    show_progress_bar=True,
    batch_size=1024
)
result = test_evaluator(model)

#{'cosine_accuracy': 0.5989669561386108}

Batches:   0%|          | 0/551 [00:00<?, ?it/s]

Batches:   0%|          | 0/551 [00:00<?, ?it/s]

Batches:   0%|          | 0/551 [00:00<?, ?it/s]

In [32]:
result

{'cosine_accuracy': 0.5944690704345703}

In [ ]:
model.save_pretrained("./last_one")


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()